In [32]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/group-chat-data/WhatsApp Chat with IITR MT(Pre-Final year) 22-23.txt
/kaggle/input/group-chat-data/stop_hinglish.txt


## preprocessor 

In [33]:
pip install pandas wordcloud emoji matplotlib seaborn nltk gradio urlextract 


Note: you may need to restart the kernel to use updated packages.


In [34]:
p ='/kaggle/input/group-chat-data/WhatsApp Chat with IITR MT(Pre-Final year) 22-23.txt'

In [35]:
with open(p, 'r', encoding='utf-8') as file:
        lines = file.readlines() 

In [52]:
import pandas as pd
import re
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import emoji
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk

# Download NLTK stopwords
nltk.download('stopwords')
nltk.download('punkt')

# Function to parse WhatsApp chat
def parse_chat(content):
    # with open(file_path, 'r', encoding='utf-8') as file:
    #     lines = file.readlines()
    lines = content
    data = []
    chat_pattern = r'^(\d{1,2}/\d{1,2}/\d{2,4}), (\d{1,2}:\d{2}\s?[APap][Mm]) - ([^:]+): (.+)$'

    for line in lines:
        match = re.match(chat_pattern, line)
        if match:
            date, time, sender, message = match.groups()
            data.append([date, time, sender, message])
        else:
            # Append message continuation to the last entry if it's not a new message
            if data and len(line.strip()) > 0:
                data[-1][3] += " " + line.strip()
    
    chat_df = pd.DataFrame(data, columns=['Date', 'Time', 'Sender', 'Message'])
    # Drop system messages
    system_messages = [
        "Messages and calls are end-to-end encrypted.",
        "This message was deleted"
    ]
    chat_df = chat_df[~chat_df['Message'].isin(system_messages)]

    # Load Hinglish stopwords
    hinglish_stopwords_path ='/kaggle/input/group-chat-data/stop_hinglish.txt'
    with open(hinglish_stopwords_path, 'r', encoding='utf-8') as file:
        hinglish_stopwords = file.read().splitlines()

    # Combine Hinglish stopwords with English stopwords
    custom_stopwords = stopwords.words('english') + hinglish_stopwords + ['Media', 'media', 'omitted']

    def remove_stopwords(text):
        tokens = word_tokenize(text)
        return ' '.join(word for word in tokens if word.lower() not in custom_stopwords and word.isalpha())
    
    # Preprocess messages
    def extract_emojis(text):
        return ''.join(c for c in text if c in emoji.EMOJI_DATA)
       
    chat_df['Emojis'] = chat_df['Message'].apply(extract_emojis)
    chat_df['Cleaned_Message'] = chat_df['Message'].apply(remove_stopwords)
    # Active hours
    chat_df['Hour'] = pd.to_datetime(chat_df['Time'], format='%I:%M %p').dt.hour
    hourly_freq = chat_df['Hour'].value_counts().sort_index() 

    chat_df['Datetime'] = pd.to_datetime(chat_df['Date'] + ' ' + chat_df['Time'], format='%m/%d/%y %I:%M %p')
    chat_df['DayOfWeek'] = chat_df['Datetime'].dt.day_name()
    chat_df['Month'] = chat_df['Datetime'].dt.month_name()
    chat_df['Year'] = chat_df['Datetime'].dt.year
    chat_df.drop(columns=['Datetime'], inplace=True)
    return chat_df


[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [53]:
# chat_df = parse_chat(lines)
# chat_df 

In [54]:
chat_df['Month'].unique() 

array(['October', 'November', 'December', 'January', 'February', 'March',
       'April', 'May', 'June', 'July', 'August', 'September'],
      dtype=object)

In [55]:
# Function to find the most active senders
def most_active_senders(chat_df, specific_sender=None):
    if specific_sender:
        chat_df = chat_df[chat_df['Sender'] == specific_sender]
    top_senders = chat_df['Sender'].value_counts().head(10)
    plt.figure(figsize=(10, 8))
    top_senders.plot(kind='bar', title="Top 10 Senders")
    plt.xlabel('Sender')
    plt.ylabel('Number of Messages')
    plt.show()
    return plt.gcf()

# Function to analyze message frequency over time
def messages_over_time(chat_df, specific_sender=None):
    if specific_sender:
        chat_df = chat_df[chat_df['Sender'] == specific_sender]
    chat_df['Datetime'] = pd.to_datetime(chat_df['Date'] + ' ' + chat_df['Time'], format='%m/%d/%y %I:%M %p')
    messages_over_time = chat_df.set_index('Datetime').resample('M').size()
    plt.figure(figsize=(10, 6))
    messages_over_time.plot(title="Messages Over Time")
    plt.xlabel('Month')
    plt.ylabel('Number of Messages')
    plt.show()
    chat_df.drop(columns=['Datetime'], inplace=True)
    return plt.gcf()

# Function to determine active hours
def active_hours(chat_df, specific_sender=None):
    if specific_sender:
        chat_df = chat_df[chat_df['Sender'] == specific_sender]
    hourly_freq = chat_df['Hour'].value_counts().sort_index()
    plt.figure(figsize=(10, 6))
    sns.barplot(x=hourly_freq.index, y=hourly_freq.values, palette='viridis')
    plt.title("Messages by Hour")
    plt.xlabel('Hour of the Day')
    plt.ylabel('Number of Messages')
    plt.show()
    return plt.gcf()

# Function to generate a word cloud of the most common words
def common_words_wordcloud(chat_df, specific_sender=None):
    if specific_sender:
        chat_df = chat_df[chat_df['Sender'] == specific_sender]
    all_words = ' '.join(chat_df['Cleaned_Message'])
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate(all_words)
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.show()
    return plt.gcf()

# Function to identify the most frequently used emojis
def emoji_analysis(chat_df, specific_sender=None):
    if specific_sender:
        chat_df = chat_df[chat_df['Sender'] == specific_sender]
    emoji_list = ''.join(chat_df['Emojis'])
    emoji_freq = Counter(emoji_list)
    most_common_emojis = emoji_freq.most_common(10)
    emoji_df = pd.DataFrame(most_common_emojis, columns=['Emoji', 'Count'])
    plt.figure(figsize=(10, 6))
    sns.barplot(x=emoji_df['Emoji'], y=emoji_df['Count'])
    plt.title("Most Common Emojis")
    plt.xlabel('Emoji')
    plt.ylabel('Count')
    plt.show()
    return plt.gcf()


# Function to get the number of messages and word lengths
def messages_and_word_lengths(chat_df, specific_sender=None):
    if specific_sender:
        chat_df = chat_df[chat_df['Sender'] == specific_sender]
    num_messages = len(chat_df)
    word_lengths = chat_df['Message'].apply(lambda x: len(x.split())).sum()
    return f"Number of Messages: {num_messages}\nTotal Word Length: {word_lengths}"

In [56]:
# Gradio interface
import gradio as gr
def analyze_chat(file, analysis_type, analysis_scope, specific_sender=None):
    with open(file, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    chat_df = parse_chat(lines)
    
    if analysis_scope == 'Specific Sender' and specific_sender:
        chat_df = chat_df[chat_df['Sender'] == specific_sender]
    
    if analysis_type == 'Most Active Senders':
        return most_active_senders(chat_df)
    elif analysis_type == 'Messages Over Time':
        return messages_over_time(chat_df)
    elif analysis_type == 'Active Hours':
        return active_hours(chat_df)
    elif analysis_type == 'Common Words Word Cloud':
        return common_words_wordcloud(chat_df)
    elif analysis_type == 'Emoji Analysis':
        return emoji_analysis(chat_df)
    # elif analysis_type == 'Number of Messages and Word Lengths':
    #     return messages_and_word_lengths(chat_df)
    else:
        return "Invalid analysis type selected."


def get_senders(file):
      with open(file, 'r', encoding='utf-8') as f:
          lines = f.readlines()
      chat_df = parse_chat(lines)
      senders = chat_df['Sender'].unique()
      return sorted(senders)

# Gradio interface
with gr.Blocks() as interface:
    file = gr.File(label="Upload WhatsApp Chat File (.txt)")
    analysis_type = gr.Dropdown(choices=['Most Active Senders', 'Messages Over Time', 'Active Hours', 'Common Words Word Cloud', 'Emoji Analysis'], label="Select Analysis Type")
    analysis_scope = gr.Radio(choices=['Overall', 'Specific Sender'], label="Analysis Scope")
    specific_sender = gr.Dropdown(label="Select Specific Sender (if applicable)")

    def update_senders(file):
        senders = get_senders(file)
        return gr.update(choices=senders)

    file.change(update_senders, file, specific_sender)
    
    analyze_button = gr.Button("Analyze")
    output = gr.Plot()
    
    def on_analyze(file, analysis_type, analysis_scope, specific_sender):
        if analysis_scope == 'Specific Sender' and not specific_sender:
            return "Please select a specific sender."
        return analyze_chat(file, analysis_type, analysis_scope, specific_sender)

    analyze_button.click(on_analyze, inputs=[file, analysis_type, analysis_scope, specific_sender], outputs=output)

interface.launch(share=True) 

* Running on local URL:  http://127.0.0.1:7872
* Running on public URL: https://4ed6654feaebceaa05.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
